In [ ]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

client = OpenAI(api_key="sk-")

In [ ]:
default_system_message = "당신은 사용자의 질문에 답을 충실히 찾아내는 유능한 비서입니다."
def get_chatgpt_response(system_message = default_system_message, user_message = "", template=0.8):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        max_tokens=16384,
        n=1,
        stream=False,
        temperature=template,
        messages = messages
    ),

    return response[0].choices[0].message.content

# Few shot engineering

### 영어를 한국어 번역하는 5-shot prompt 를 작성하고 "dog"를 번역해보세요.

In [ ]:
system_message = f"""
Translate the following words to Korean:
1. book: 책
2. cheese: 치즈
3. water: 물
4. bread: 빵
5. apple: 사과
"""

user_message = f"""
Translate the word "dog":
"""

get_chatgpt_response(system_message = system_message, user_message = user_message)

영화 리뷰에 대한 sentiment (positive or negative) 를 결정하는 5-shot prompt 를 작성하고 "The storyline was dull and uninspiring." 에 대한 결과를 확인해보세요.

In [ ]:
system_message = f"""
Determine the sentiment (positive or negative) of the following movie reviews:
1. "The movie was fantastic and had great acting.": positive
2. "I hated the film; it was a complete waste of time.": negative
3. "An excellent film that kept me on the edge of my seat.": positive
4. "Terrible plot and horrible acting.": negative
5. "A wonderful experience with stunning visuals.": positive
"""

user_message = f"""
Review: "The storyline was dull and uninspiring."
Sentiment:
"""

get_chatgpt_response(system_message = system_message, user_message = user_message)

자연어를 SQL 쿼리로 바꿔주는 few-shot prompt 를 작성해보세요.

In [ ]:
system_message = f"""
Convert the following natural language requests into SQL queries:
1. "Show all employees with a salary greater than $50,000.": SELECT * FROM employees WHERE salary > 50000;
2. "List all products that are out of stock.": SELECT * FROM products WHERE stock = 0;
3. "Find the names of students who scored above 90 in math.": SELECT name FROM students WHERE math_score > 90;
4. "Retrieve the details of orders placed in the last 30 days.": SELECT * FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY);
5. "Get the count of customers from each city.": SELECT city, COUNT(*) FROM customers GROUP BY city;
"""

user_message = f"""
Request: "Find the average salary of employees in the marketing department."
SQL Query:
"""

get_chatgpt_response(system_message = system_message, user_message = user_message)



---



# Chain of Thought

### 다음 문제를 푸는 Chain of thought prompt 를 작성해보세요.

Solve the following problem step-by-step: 23 + 47

Step-by-step solution:

1. Break down the numbers: 23 and 47
2. Add the tens place: 20 + 40 = 60
3. Add the ones place: 3 + 7 = 10
4. Combine the results: 60 + 10 = 70

Answer: 70

In [ ]:
system_message = f"""
Solve the following arithmetic problem step-by-step using the Chain of Thought (CoT) technique:

Problem: 23 + 47

Step-by-step solution:

Break down the numbers: 23 and 47.
Add the tens place: 20 + 40 = 60.
Add the ones place: 3 + 7 = 10.
Combine the results: 60 + 10 = 70.

Answer: 70

Use this step-by-step method to solve other arithmetic problems provided.
"""

user_message = f"""
123 - 58
"""

result = get_chatgpt_response(system_message = system_message, user_message = user_message, template=0.2)
print(result)

In [ ]:
system_message = f"""
Solve the following arithmetic problem step-by-step using the Chain of Thought (CoT) technique:

Problem: 23 + 47

Step-by-step solution:

Break down the numbers: 23 and 47.
Add the tens place: 20 + 40 = 60.
Add the ones place: 3 + 7 = 10.
Combine the results: 60 + 10 = 70.

Answer: 70

Use this step-by-step method to solve other arithmetic problems provided.
"""

user_message = f"""
345 + 678 - 123
"""

result = get_chatgpt_response(system_message = system_message, user_message = user_message, template=0.2)
print(result)

### 다음 문제를 푸는 Chain of thought prompt 를 작성해보세요.

Solve the following logic puzzle step-by-step:
Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:
1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.

Determine the favorite color of each friend.

Step-by-step solution:
1. Analyze the information:
   a. Carol likes green (given).
   b. Alice does not like red.
   c. Bob does not like blue.
2. Since Carol likes green, Alice and Bob cannot like green.
3. Alice does not like red, so Alice must like blue.
4. Bob does not like blue and cannot like green, so Bob must like red.
5. Verify:
   a. Carol likes green.
   b. Alice likes blue.
   c. Bob likes red.

Answer:
- Alice: blue
- Bob: red
- Carol: green

In [ ]:
system_message = f"""
**System Instruction:** You are an AI designed to solve logic puzzles using the Chain of Thought method. When given a puzzle, provide a detailed, logical step-by-step explanation for each deduction you make to reach the correct solution. Ensure your reasoning is clear and follows logically from the given information.

**Input:**
A logic puzzle with specific constraints and clues.

**Output:**
Provide a step-by-step solution using the Chain of Thought method, explaining the reasoning behind each step, and then summarize the final answer.

**Example Format:**

**Step-by-step solution:**
1. **Analyze the information:** Identify and list all the constraints and clues provided in the puzzle.
2. **Initial deductions:** Use the given information to make initial logical deductions.
3. **Narrow down possibilities:** Based on the initial deductions, eliminate impossible options for each variable (e.g., favorite color, person).
4. **Further deductions:** Make further logical deductions based on the remaining possibilities.
5. **Verify consistency:** Ensure that all deductions are consistent with the given constraints and clues.
6. **Finalize the solution:** Summarize the final answer, ensuring all parts of the puzzle are addressed.

**Instructions for AI:**
1. Read and understand the given puzzle.
2. Identify and list all the constraints and clues.
3. Deduce the solution step-by-step, ensuring each step logically follows from the previous one.
4. Eliminate impossibilities and narrow down options logically.
5. Verify consistency at each step.
6. Summarize the final answer, clearly matching each variable with its determined value.
7. Ensure clarity and logical consistency throughout the explanation.


**Example:**

**Puzzle:**
Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:
1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.
Determine the favorite color of each friend.

**Step-by-step solution:**
1. Analyze the information:
Carol likes green (given).
Alice does not like red.
Bob does not like blue.

2. Initial deductions:
Since Carol likes green, Alice and Bob cannot like green.

3. Narrow down possibilities:
Alice has two possible colors left: blue and green. Since green is already taken by Carol, Alice must like blue.
Bob has two possible colors left: red and green. Since green is already taken by Carol and Bob does not like blue, Bob must like red.

4. Further deductions:
All deductions align with the given constraints.

5. Verify consistency:
Carol likes green.
Alice likes blue.
Bob likes red.

6. Finalize the solution:
Alice: blue
Bob: red
Carol: green
"""

user_message = f"""
Solve the following logic puzzle step-by-step: Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:

Alice does not like red.
Bob does not like blue.
Carol likes green.
Determine the favorite color of each friend.
"""

result = get_chatgpt_response(system_message = system_message, user_message = user_message, template=0.2)
print(result)

**Step-by-step solution:**

1. **Analyze the information:**
   - We have three friends: Alice, Bob, and Carol.
   - Their favorite colors are red, blue, and green.
   - The clues provided are:
     - Alice does not like red.
     - Bob does not like blue.
     - Carol likes green.

2. **Initial deductions:**
   - From the clue that Carol likes green, we can immediately assign green to Carol. 
   - This means Alice and Bob cannot like green.

3. **Narrow down possibilities:**
   - Since Carol likes green, the remaining colors for Alice and Bob are red and blue.
   - According to the clue, Alice does not like red. Therefore, the only color left for Alice is blue.
   - This leaves Bob with the only remaining color, which is red.

4. **Further deductions:**
   - We have now assigned:
     - Carol: green
     - Alice: blue
     - Bob: red
   - These assignments do not contradict any of the clues provided.

5. **Verify consistency:**
   - Check each friend against the clues:
     - Alice doe

### 다음 문제를 푸는 Chain of thought prompt 를 작성해보세요.

Solve the following logic puzzle step-by-step:
Four people (A, B, C, D) are sitting in a row. We know that:
1. A is not next to B.
2. B is next to C.
3. C is not next to D.

Determine the possible seating arrangements.

Step-by-step solution:
1. Analyze the information:
   a. A is not next to B.
   b. B is next to C.
   c. C is not next to D.
2. List possible positions for B and C:
   a. B and C must be next to each other.
   b. Possible pairs: BC, CB.
3. Determine positions for A and D:
   a. If B is first (BCA, BCAD), then A cannot be next to B.
   b. If C is first (CAB, CABD), then D cannot be next to C.
4. Combine information to find valid arrangements:
   a. BCAD: B is next to C, A is not next to B, C is not next to D.
   b. CABD: B is next to C, A is not next to B, C is not next to D.

Answer:
- Possible arrangements: BCAD, CABD

In [ ]:
system_message = f"""
**System Instruction:** You are an AI designed to solve logic puzzles using the Chain of Thought method. When given a puzzle, provide a detailed, logical step-by-step explanation for each deduction you make to reach the correct solution. Ensure your reasoning is clear and follows logically from the given information.

**Input:**
A logic puzzle with specific constraints and clues.

**Output:**
Provide a step-by-step solution using the Chain of Thought method, explaining the reasoning behind each step, and then summarize the final answer.

**Example Format:**

**Step-by-step solution:**
1. **Analyze the information:** Identify and list all the constraints and clues provided in the puzzle.
2. **Initial deductions:** Use the given information to make initial logical deductions.
3. **Narrow down possibilities:** Based on the initial deductions, eliminate impossible options for each variable (e.g., favorite color, person).
4. **Further deductions:** Make further logical deductions based on the remaining possibilities.
5. **Verify consistency:** Ensure that all deductions are consistent with the given constraints and clues.
6. **Finalize the solution:** Summarize the final answer, ensuring all parts of the puzzle are addressed.

**Instructions for AI:**
1. Read and understand the given puzzle.
2. Identify and list all the constraints and clues.
3. Deduce the solution step-by-step, ensuring each step logically follows from the previous one.
4. Eliminate impossibilities and narrow down options logically.
5. Verify consistency at each step.
6. Summarize the final answer, clearly matching each variable with its determined value.
7. Ensure clarity and logical consistency throughout the explanation.


**Example:**

**Puzzle:**
Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:
1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.
Determine the favorite color of each friend.

**Step-by-step solution:**
1. Analyze the information:
Carol likes green (given).
Alice does not like red.
Bob does not like blue.

2. Initial deductions:
Since Carol likes green, Alice and Bob cannot like green.

3. Narrow down possibilities:
Alice has two possible colors left: blue and green. Since green is already taken by Carol, Alice must like blue.
Bob has two possible colors left: red and green. Since green is already taken by Carol and Bob does not like blue, Bob must like red.

4. Further deductions:
All deductions align with the given constraints.

5. Verify consistency:
Carol likes green.
Alice likes blue.
Bob likes red.

6. Finalize the solution:
Alice: blue
Bob: red
Carol: green
"""

user_message = f"""
Solve the following logic puzzle step-by-step: Four people (A, B, C, D) are sitting in a row. We know that:

A is not next to B.
B is next to C.
C is not next to D.
Determine the possible seating arrangements.
"""

result = get_chatgpt_response(system_message = system_message, user_message = user_message, template=0.2)
print(result)

**Step-by-step solution:**

1. **Analyze the information:**
   - We have four people: A, B, C, and D.
   - The constraints are:
     1. A is not next to B.
     2. B is next to C.
     3. C is not next to D.

2. **Initial deductions:**
   - From the second constraint, since B is next to C, we can represent them as a block (BC) or (CB).
   - This means that B and C must occupy adjacent positions in the row.

3. **Narrow down possibilities:**
   - The possible arrangements for B and C as a block are:
     - (B, C)
     - (C, B)
   - Given that there are four positions, we can place the BC block in the following ways:
     1. Positions 1 and 2: (B, C, _, _)
     2. Positions 2 and 3: (_, B, C, _)
     3. Positions 3 and 4: (_, _, B, C)
     4. Positions 1 and 2: (C, B, _, _)
     5. Positions 2 and 3: (_, C, B, _)
     6. Positions 3 and 4: (_, _, C, B)

4. **Further deductions:**
   - Now we need to place A and D in the remaining positions while respecting the constraints.
   - Let's ana

---